In [2]:
import os
import random
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from tqdm import tqdm

将图像和标签对应起来

In [3]:
df_test=pd.DataFrame()
df_test['img']=''
df_test['label']=''
df_train=pd.DataFrame()
df_train['img']=''
df_train['label']=''
idx1=0
idx2=0
try:
    os.system('unzip RockData')
    os.system('mkdir imgdata')
    list_rock=os.listdir('RockData')
    print(list_rock)
except:
    print('无需创建')
for i in list_rock:
    pathx='RockData/'+i
    os.system(f'cp {pathx}/* imgdata')
    list_label=os.listdir(pathx)
    random.shuffle(list_label)#打乱图片顺序
    for j in list_label[:int(len(list_label)/5)]:
        df_test.loc[idx1]=[j,i]
        idx1+=1
    for j in list_label[int(len(list_label)/5):]:
        df_train.loc[idx2]=[j,i]
        idx2+=1
try:
    os.system('rm test.csv')#删掉可能存在的同名文件
    os.system('rm train.csv')
except:
    pass
df_test.to_csv('test.csv')
df_train.to_csv('train.csv')

Archive:  RockData.zip
   creating: RockData/
   creating: RockData/Basalt/
  inflating: RockData/Basalt/Basalt1.jpg  
  inflating: RockData/Basalt/Basalt10.jpg  
  inflating: RockData/Basalt/Basalt11.jpg  
  inflating: RockData/Basalt/Basalt12.jpg  
  inflating: RockData/Basalt/Basalt13.jpg  
  inflating: RockData/Basalt/Basalt14.jpg  
  inflating: RockData/Basalt/Basalt15.jpg  
  inflating: RockData/Basalt/Basalt16.jpg  
  inflating: RockData/Basalt/Basalt17.jpg  
  inflating: RockData/Basalt/Basalt18.jpg  
  inflating: RockData/Basalt/Basalt19.jpg  
  inflating: RockData/Basalt/Basalt21.jpg  
  inflating: RockData/Basalt/Basalt23.jpg  
  inflating: RockData/Basalt/Basalt25.jpg  
  inflating: RockData/Basalt/Basalt26.jpg  
  inflating: RockData/Basalt/Basalt27.jpg  
  inflating: RockData/Basalt/Basalt3.jpg  
  inflating: RockData/Basalt/Basalt30.jpg  
  inflating: RockData/Basalt/Basalt31.jpg  
  inflating: RockData/Basalt/Basalt32.jpg  
  inflating: RockData/Basalt/Basalt33.jpg  
  

  inflating: RockData/Limestone/Limestone43.jpg  
  inflating: RockData/Limestone/Limestone44.jpg  
  inflating: RockData/Limestone/Limestone45.jpg  
  inflating: RockData/Limestone/Limestone46.jpg  
  inflating: RockData/Limestone/Limestone47.jpg  
  inflating: RockData/Limestone/Limestone48.jpg  
  inflating: RockData/Limestone/Limestone49.jpg  
  inflating: RockData/Limestone/Limestone5.jpg  
  inflating: RockData/Limestone/Limestone50.jpg  
  inflating: RockData/Limestone/Limestone51.jpg  
  inflating: RockData/Limestone/Limestone52.jpg  
  inflating: RockData/Limestone/Limestone53.jpg  
  inflating: RockData/Limestone/Limestone54.jpg  
  inflating: RockData/Limestone/Limestone55.jpg  
  inflating: RockData/Limestone/Limestone56.jpg  
  inflating: RockData/Limestone/Limestone57.jpg  
  inflating: RockData/Limestone/Limestone58.jpg  
  inflating: RockData/Limestone/Limestone59.jpg  
  inflating: RockData/Limestone/Limestone6.jpg  
  inflating: RockData/Limestone/Limestone60.jpg  
  

  inflating: RockData/Sandstone/Sandstone244.jpg  
  inflating: RockData/Sandstone/Sandstone245.jpg  
  inflating: RockData/Sandstone/Sandstone246.jpg  
  inflating: RockData/Sandstone/Sandstone247.jpg  
  inflating: RockData/Sandstone/Sandstone248.jpg  
  inflating: RockData/Sandstone/Sandstone249.jpg  
  inflating: RockData/Sandstone/Sandstone25.jpg  
  inflating: RockData/Sandstone/Sandstone250.jpg  
  inflating: RockData/Sandstone/Sandstone251.jpg  
  inflating: RockData/Sandstone/Sandstone252.jpg  
  inflating: RockData/Sandstone/Sandstone253.jpg  
  inflating: RockData/Sandstone/Sandstone254.jpg  
  inflating: RockData/Sandstone/Sandstone255.jpg  
  inflating: RockData/Sandstone/Sandstone256.jpg  
  inflating: RockData/Sandstone/Sandstone257.jpg  
  inflating: RockData/Sandstone/Sandstone258.jpg  
  inflating: RockData/Sandstone/Sandstone259.jpg  
  inflating: RockData/Sandstone/Sandstone26.jpg  
  inflating: RockData/Sandstone/Sandstone260.jpg  
  inflating: RockData/Sandstone/S

rm: cannot remove 'test.csv': No such file or directory
rm: cannot remove 'train.csv': No such file or directory


标签字符串与数字间的对应

In [4]:
class_to_num = dict(zip(list_rock, range(len(list_rock))))
class_to_num

{'Coal': 0,
 'Marble': 1,
 'Granite': 2,
 'Limestone': 3,
 'Sandstone': 4,
 'Quartzite': 5,
 'Basalt': 6}

In [5]:
num_to_class = {v : k for k, v in class_to_num.items()}
num_to_class

{0: 'Coal',
 1: 'Marble',
 2: 'Granite',
 3: 'Limestone',
 4: 'Sandstone',
 5: 'Quartzite',
 6: 'Basalt'}

In [6]:
df_train.head()

,img,label
0,Coal29.jpg,Coal
1,Coal158.jpg,Coal
2,Coal312.jpg,Coal
3,Coal208.jpg,Coal
4,Coal22.jpg,Coal


In [7]:
df_test.tail()

,img,label
403,Basalt25.jpg,Basalt
404,Basalt19.jpg,Basalt
405,Basalt8.jpg,Basalt
406,Basalt68.jpg,Basalt
407,Basalt62.jpg,Basalt


In [8]:
data_info = pd.read_csv('train.csv', header=None)  
data_info.head()

,0,1,2
0,NaN,img,label
1,0.0,Coal29.jpg,Coal
2,1.0,Coal158.jpg,Coal
3,2.0,Coal312.jpg,Coal
4,3.0,Coal208.jpg,Coal


In [9]:
class RockData(Dataset):
    def __init__(self, csv_path, file_path, mode='train', valid_ratio=0.2, resize_height=256, resize_width=256):
        
        # 图像高宽
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.mode = mode

    
        self.data_info = pd.read_csv(csv_path, header=None)  
        self.data_len = len(self.data_info.index) - 1
        self.train_len = int(self.data_len * (1 - valid_ratio))
        
        if mode == 'train':
            # 第一列包含图像文件的名称
            self.train_image = np.asarray(self.data_info.iloc[1:self.train_len, [1]])  
            # 第二列是图像的 label
            self.train_label = np.asarray(self.data_info.iloc[1:self.train_len, [2]])
            self.image_arr = self.train_image 
            self.label_arr = self.train_label
        elif mode == 'valid':
            self.valid_image = np.asarray(self.data_info.iloc[self.train_len:, [1]])  
            self.valid_label = np.asarray(self.data_info.iloc[self.train_len:, [2]])
            self.image_arr = self.valid_image
            self.label_arr = self.valid_label
        elif mode == 'test':
            self.test_image = np.asarray(self.data_info.iloc[1:, [1]])
            self.image_arr = self.test_image
            
        self.real_len = len(self.image_arr)

        print('Finished reading the {} set of Leaves Dataset ({} samples found)'.format(mode, self.real_len))

    def __getitem__(self, index):
        single_image_name = self.image_arr[index][0]
   
        pathx=self.file_path + single_image_name
        try:
            #pathx=pathx[0]
            #index np.array
            img_as_img = Image.open(pathx)
            #其实也可以用opencv，只不过opencv是BGR,但是低版本的PIL只能读取Image，不能读取Numpy，高版本读取Numpy会报warning
            #https://zhuanlan.zhihu.com/p/195700420
            #print(pathx)
        except:
            
            print(pathx)
            
        img_as_img = img_as_img.convert("RGB")

        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        if self.mode == 'train':
            transform = transforms.Compose([
                transforms.Resize((224, 224)),#只能resize高宽，不能改变通道
                #图像增广
                transforms.RandomHorizontalFlip(p=0.5),   #随机水平翻转 选择一个概率
                transforms.ToTensor()#转换为torch的张量，当然这里如果用opnecv也可以转换为numpy,dans
            ])
        else:
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
        
        img_as_img = transform(img_as_img)
        
        if self.mode == 'test':
            return img_as_img
        else:
            label = self.label_arr[index][0]
            number_label = class_to_num[label]

            return img_as_img, number_label  

    def __len__(self):
        return self.real_len


In [10]:
train_path = 'train.csv'
test_path = 'test.csv'
img_path = 'imgdata/'

train_dataset = RockData(train_path, img_path, mode='train')
val_dataset = RockData(train_path, img_path, mode='valid')
test_dataset = RockData(test_path, img_path, mode='test')
print(train_dataset)
print(val_dataset)
print(test_dataset)

Finished reading the train set of Leaves Dataset (1313 samples found)
Finished reading the valid set of Leaves Dataset (330 samples found)
Finished reading the test set of Leaves Dataset (408 samples found)


In [11]:
train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,#选择数据集
        batch_size=8, #定义批量
        shuffle=False,#不使用随机
        num_workers=5#工作线程
    )

val_loader = torch.utils.data.DataLoader(
        dataset=val_dataset,
        batch_size=8, 
        shuffle=False,
        num_workers=5
    )
test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=8, 
        shuffle=False,
        num_workers=5
    )

处理到GPU上

In [12]:
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [13]:
# 冻结参数以节省时间
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False
# resnet34模型
def res_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.resnet34(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [14]:
#超参数
learning_rate = 1e-3
weight_decay = 1e-3
num_epoch = 5
model_path = './smartoil.ckpt'#模型保存文件

In [15]:
model = res_model(7)
model = model.to(device)
model.device = device

#交叉熵损失
criterion = nn.CrossEntropyLoss()
#Adam优化器，对初始值不敏感
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
#训练周期
n_epochs = num_epoch

best_acc = 0.0
for epoch in range(n_epochs):
    model.train() 
    train_loss = []
    train_accs = []
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs = imgs.to(device)# to cuda
        labels = labels.to(device)
        logits = model(imgs)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (logits.argmax(dim=-1) == labels).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    model.eval()
    
    valid_loss = []
    valid_accs = []

    for batch in tqdm(val_loader):
        imgs, labels = batch
 
        with torch.no_grad():
            logits = model(imgs.to(device))
            
        loss = criterion(logits, labels.to(device))

        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    if valid_acc > best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc))

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/upc/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

100%|█████████████████████████████████████████| 165/165 [00:07<00:00, 21.64it/s]


[ Train | 001/005 ] loss = 1.91232, acc = 0.60606


100%|███████████████████████████████████████████| 42/42 [00:00<00:00, 60.83it/s]


...

[ Valid | 004/005 ] loss = 126.36623, acc = 0.41071
saving model with acc 0.411


100%|█████████████████████████████████████████| 165/165 [00:07<00:00, 22.65it/s]


[ Train | 005/005 ] loss = 1.47515, acc = 0.53788


100%|███████████████████████████████████████████| 42/42 [00:00<00:00, 59.87it/s]

[ Valid | 005/005 ] loss = 2.11866, acc = 0.00000


In [16]:
saveFileName = './submission.csv'

In [17]:
model = res_model(7)

model = model.to(device)
model.load_state_dict(torch.load(model_path))#使用训练好的参数

model.eval()#调整为评价模式

predictions = []
for batch in tqdm(test_loader):
    
    imgs = batch
    with torch.no_grad():
        logits = model(imgs.to(device))
    
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())#加入新列表

preds = []
for i in predictions:
    preds.append(num_to_class[i])

test_data = pd.read_csv(test_path)
test_data['label'] = pd.Series(preds)
submission = pd.concat([test_data['img'], test_data['label']], axis=1)
submission.to_csv(saveFileName, index=False)

100%|███████████████████████████████████████████| 51/51 [00:00<00:00, 61.23it/s]


right